#### Imports

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial import distance
from matplotlib import pyplot as plt
from itertools import permutations
import glob
import csv
import os
import re
import json


#### Get Data

In [2]:
WPA_morning_testing_df = pd.read_csv('../clean data/WPA_morning_testing.csv')
WPA_night_training_df = pd.read_csv('../clean data/WPA_night_training.csv')

display(WPA_morning_testing_df)


,Unnamed: 0,id,user_id,user,test_type,wpa_morning_testing,started_at,finished_at
0,0,64a9ab8ba23f348e35502d65,6423244261f11182a22e06c7,rtocs_999_6,Morning Testing,"[{'wordPair': ['truce', 'lobe'], 'swapped': Tr...",1688840985503,1688841099654
1,1,64ad870b57a28b868459666b,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,"[{'wordPair': ['blimp', 'bingo'], 'swapped': F...",1689093852613,1689093899010
2,2,64ac54f0516ec0ee389d8369,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,"[{'wordPair': ['group', 'blush'], 'swapped': F...",1689015479136,1689015536528
3,3,64aac8e2133b98be33469f21,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,"[{'wordPair': ['edgy', 'salve'], 'swapped': Tr...",1688914100808,1688914145625
4,4,64a9902b9a987115adbff6a4,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,"[{'wordPair': ['alive', 'nacho'], 'swapped': F...",1688834039852,1688834091126
5,5,64a84ec28990048aa6d54f5b,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,"[{'wordPair': ['mumbo', 'west'], 'swapped': Fa...",1688751762329,1688751809458
6,6,64a708c94f405d0e60bd992c,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,"[{'wordPair': ['yummy', 'scalp'], 'swapped': T...",1688668314828,1688668361708


#### Inspect Columns

In [3]:
morning_tests = WPA_morning_testing_df.loc[:, 'wpa_morning_testing']

night_tests = WPA_night_training_df.loc[:, 'wpa_night_testing']
night_training_pairs = WPA_night_training_df.loc[:, 'wpa_training']    # No need to mess with this

display(morning_tests)
display(night_tests)


0    [{'wordPair': ['truce', 'lobe'], 'swapped': Tr...
1    [{'wordPair': ['blimp', 'bingo'], 'swapped': F...
2    [{'wordPair': ['group', 'blush'], 'swapped': F...
3    [{'wordPair': ['edgy', 'salve'], 'swapped': Tr...
4    [{'wordPair': ['alive', 'nacho'], 'swapped': F...
5    [{'wordPair': ['mumbo', 'west'], 'swapped': Fa...
6    [{'wordPair': ['yummy', 'scalp'], 'swapped': T...
Name: wpa_morning_testing, dtype: object

0    [{'wordPair': ['crow', 'sage'], 'swapped': Tru...
1    [{'wordPair': ['cram', 'bugle'], 'swapped': Tr...
2    [{'wordPair': ['story', 'knack'], 'swapped': F...
3    [{'wordPair': ['moxie', 'sassy'], 'swapped': T...
4    [{'wordPair': ['deft', 'sharp'], 'swapped': Fa...
5    [{'wordPair': ['nacho', 'five'], 'swapped': Tr...
6    [{'wordPair': ['risk', 'shout'], 'swapped': Fa...
7    [{'wordPair': ['jade', 'type'], 'swapped': Fal...
Name: wpa_night_testing, dtype: object

#### Make New DataFrame for Testing

In [4]:
def word_pairs_df(initial_df, subjects: int):
    np_array = []
    columns = []

    for sample_index in range(subjects):

        # Put into readable json format before conversion.
        word_pairs_str = initial_df[sample_index].replace("'", '"')
        word_pairs_str = word_pairs_str.replace('True', 'true')
        word_pairs_str = word_pairs_str.replace('False', 'false')
        
        # Convert using json.
        word_pairs_list = json.loads(word_pairs_str)
        
        for i in range(len(word_pairs_list)):
            if sample_index == 0:
                columns_default = list(word_pairs_list[i].keys())
                for column in columns_default:
                    columns.append(f'{column}_{i+1}')
        
        sample_values = []

        for word_pair_dict in word_pairs_list:
            sample_values.append(list(word_pair_dict.values()))
            
        np_array.append(sample_values)
        
    np_array = np.array(np_array, dtype='O')
    np_array = np_array.reshape(np_array.shape[0], -1)

    dataframe = pd.DataFrame(np_array, columns=columns)
    return dataframe

morning_tests_df = word_pairs_df(morning_tests, 7)
night_tests_df = word_pairs_df(night_tests, 8)


In [5]:
display(morning_tests_df)

,wordPair_1,swapped_1,rearranged_1,isCorrect_1,choice_1,choiceTime_1,confidence_1,confidenceTime_1,wordPair_2,swapped_2,...,confidence_19,confidenceTime_19,wordPair_20,swapped_20,rearranged_20,isCorrect_20,choice_20,choiceTime_20,confidence_20,confidenceTime_20
0,"[truce, lobe]",True,False,False,DIFFERENT,1688841054471,MODERATE,1688841055122,"[phone, bugle]",True,...,LOW,1688841098035,"[jump, lycra]",True,True,True,DIFFERENT,1688841099253,MODERATE,1688841099654
1,"[blimp, bingo]",False,False,True,SAME,1689093857142,HIGH,1689093858092,"[mango, size]",False,...,HIGH,1689093896842,"[goofy, kind]",True,True,True,DIFFERENT,1689093898676,HIGH,1689093899009
2,"[group, blush]",False,False,True,SAME,1689015483642,HIGH,1689015484242,"[moxie, flame]",True,...,HIGH,1689015534445,"[waive, yeast]",False,True,True,DIFFERENT,1689015536145,HIGH,1689015536528
3,"[edgy, salve]",True,True,True,DIFFERENT,1688914105070,HIGH,1688914105504,"[sled, hymn]",False,...,HIGH,1688914143673,"[coin, site]",False,True,True,DIFFERENT,1688914145224,HIGH,1688914145625
4,"[alive, nacho]",False,True,True,DIFFERENT,1688834046904,HIGH,1688834047488,"[kelp, pesto]",False,...,HIGH,1688834087086,"[ping, month]",False,True,True,DIFFERENT,1688834089689,LOW,1688834091125
5,"[mumbo, west]",False,True,True,DIFFERENT,1688751767717,HIGH,1688751768150,"[vest, sonic]",False,...,LOW,1688751806909,"[deem, odor]",False,True,True,DIFFERENT,1688751808958,HIGH,1688751809458
6,"[yummy, scalp]",True,False,True,SAME,1688668319757,HIGH,1688668320207,"[stoop, gall]",True,...,HIGH,1688668360141,"[bear, trip]",True,True,True,DIFFERENT,1688668361425,HIGH,1688668361708


In [6]:
display(night_tests_df)

,wordPair_1,swapped_1,rearranged_1,isCorrect_1,choice_1,choiceTime_1,confidence_1,confidenceTime_1,wordPair_2,swapped_2,...,confidence_19,confidenceTime_19,wordPair_20,swapped_20,rearranged_20,isCorrect_20,choice_20,choiceTime_20,confidence_20,confidenceTime_20
0,"[crow, sage]",True,True,True,DIFFERENT,1689972165930,MODERATE,1689972167085,"[gumbo, print]",False,...,HIGH,1689972214272,"[jump, mild]",True,True,True,DIFFERENT,1689972214905,HIGH,1689972215371
1,"[cram, bugle]",True,True,True,DIFFERENT,1688717661913,MODERATE,1688717663262,"[name, roll]",True,...,LOW,1688717707883,"[rhino, phone]",False,True,True,DIFFERENT,1688717710068,MODERATE,1688717710452
2,"[story, knack]",False,False,True,SAME,1689053614233,HIGH,1689053614733,"[minty, sumo]",True,...,LOW,1689053651435,"[tiny, bees]",False,True,True,DIFFERENT,1689053652503,HIGH,1689053652887
3,"[moxie, sassy]",True,True,True,DIFFERENT,1688968628273,HIGH,1688968628773,"[good, fine]",False,...,HIGH,1688968662997,"[truth, page]",True,False,True,SAME,1688968664116,HIGH,1688968664533
4,"[deft, sharp]",False,True,True,DIFFERENT,1688879109810,HIGH,1688879110525,"[edgy, hose]",True,...,HIGH,1688879153646,"[yoga, fluke]",False,True,True,DIFFERENT,1688879155665,HIGH,1688879156015
5,"[nacho, five]",True,False,True,SAME,1688791251627,HIGH,1688791252377,"[drive, grip]",False,...,HIGH,1688791296700,"[alive, robot]",False,False,True,SAME,1688791297833,HIGH,1688791298267
6,"[risk, shout]",False,False,True,SAME,1688714380326,HIGH,1688714381027,"[couch, brass]",False,...,HIGH,1688714416366,"[dress, joke]",False,True,True,DIFFERENT,1688714417666,HIGH,1688714418033
7,"[jade, type]",False,False,True,SAME,1688625927104,HIGH,1688625929088,"[rider, able]",False,...,HIGH,1688625973593,"[expo, gall]",True,False,False,DIFFERENT,1688625975976,LOW,1688625976476


#### Add Columns to CSV

In [7]:
for column in list(morning_tests_df.columns):
    WPA_morning_testing_df[column] = morning_tests_df[column]

for column in list(night_tests_df.columns):
    WPA_night_training_df[column] = night_tests_df[column]

C:\Users\rohan\AppData\Local\Temp\ipykernel_37952\1081360770.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  WPA_morning_testing_df[column] = morning_tests_df[column]
C:\Users\rohan\AppData\Local\Temp\ipykernel_37952\1081360770.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  WPA_morning_testing_df[column] = morning_tests_df[column]
C:\Users\rohan\AppData\Local\Temp\ipykernel_37952\1081360770.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

#### Get rid of Prevoius Columns

In [8]:
WPA_morning_testing_df.drop(columns=['wpa_morning_testing'], inplace=True)
WPA_night_training_df.drop(columns=['wpa_night_testing'], inplace=True)

In [9]:
display(WPA_morning_testing_df)

,Unnamed: 0,id,user_id,user,test_type,started_at,finished_at,wordPair_1,swapped_1,rearranged_1,...,confidence_19,confidenceTime_19,wordPair_20,swapped_20,rearranged_20,isCorrect_20,choice_20,choiceTime_20,confidence_20,confidenceTime_20
0,0,64a9ab8ba23f348e35502d65,6423244261f11182a22e06c7,rtocs_999_6,Morning Testing,1688840985503,1688841099654,"[truce, lobe]",True,False,...,LOW,1688841098035,"[jump, lycra]",True,True,True,DIFFERENT,1688841099253,MODERATE,1688841099654
1,1,64ad870b57a28b868459666b,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,1689093852613,1689093899010,"[blimp, bingo]",False,False,...,HIGH,1689093896842,"[goofy, kind]",True,True,True,DIFFERENT,1689093898676,HIGH,1689093899009
2,2,64ac54f0516ec0ee389d8369,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,1689015479136,1689015536528,"[group, blush]",False,False,...,HIGH,1689015534445,"[waive, yeast]",False,True,True,DIFFERENT,1689015536145,HIGH,1689015536528
3,3,64aac8e2133b98be33469f21,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,1688914100808,1688914145625,"[edgy, salve]",True,True,...,HIGH,1688914143673,"[coin, site]",False,True,True,DIFFERENT,1688914145224,HIGH,1688914145625
4,4,64a9902b9a987115adbff6a4,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,1688834039852,1688834091126,"[alive, nacho]",False,True,...,HIGH,1688834087086,"[ping, month]",False,True,True,DIFFERENT,1688834089689,LOW,1688834091125
5,5,64a84ec28990048aa6d54f5b,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,1688751762329,1688751809458,"[mumbo, west]",False,True,...,LOW,1688751806909,"[deem, odor]",False,True,True,DIFFERENT,1688751808958,HIGH,1688751809458
6,6,64a708c94f405d0e60bd992c,6423241561f11182a22e06c5,rtocs_999_3,Morning Testing,1688668314828,1688668361708,"[yummy, scalp]",True,False,...,HIGH,1688668360141,"[bear, trip]",True,True,True,DIFFERENT,1688668361425,HIGH,1688668361708


In [10]:
display(WPA_night_training_df)

,Unnamed: 0,id,user_id,user,test_type,wpa_training,started_at,finished_at,wordPair_1,swapped_1,...,confidence_19,confidenceTime_19,wordPair_20,swapped_20,rearranged_20,isCorrect_20,choice_20,choiceTime_20,confidence_20,confidenceTime_20
0,0,64baedf77440b5960a1c29b6,644604ce6b909279c959fa41,rtocs_kate,Night Training,"{'pairs': [['brick', 'brew'], ['crust', 'alarm...",1689971908593,1689972215372,"[crow, sage]",True,...,HIGH,1689972214272,"[jump, mild]",True,True,True,DIFFERENT,1689972214905,HIGH,1689972215371
1,1,64a7c98e84e0bf2849b60929,6423244261f11182a22e06c7,rtocs_999_6,Night Training,"{'pairs': [['stub', 'seal'], ['ruby', 'cram'],...",1688717276456,1688717710452,"[cram, bugle]",True,...,LOW,1688717707883,"[rhino, phone]",False,True,True,DIFFERENT,1688717710068,MODERATE,1688717710452
2,2,64ace9d553a2de319e9b17f8,6423241561f11182a22e06c5,rtocs_999_3,Night Training,"{'pairs': [['tiny', 'mite'], ['java', 'cater']...",1689053368795,1689053652887,"[story, knack]",False,...,LOW,1689053651435,"[tiny, bees]",False,True,True,DIFFERENT,1689053652503,HIGH,1689053652887
3,3,64ab9dd90e51fb55283e394f,6423241561f11182a22e06c5,rtocs_999_3,Night Training,"{'pairs': [['good', 'fine'], ['lapse', 'loop']...",1688968381590,1688968664533,"[moxie, sassy]",True,...,HIGH,1688968662997,"[truth, page]",True,False,True,SAME,1688968664116,HIGH,1688968664533
4,4,64aa4034cffc27aac7e8d756,6423241561f11182a22e06c5,rtocs_999_3,Night Training,"{'pairs': [['grass', 'crew'], ['coin', 'flood'...",1688878861843,1688879156016,"[deft, sharp]",False,...,HIGH,1688879153646,"[yoga, fluke]",False,True,True,DIFFERENT,1688879155665,HIGH,1688879156015
5,5,64a8e9023a6ffcbbaeeaba75,6423241561f11182a22e06c5,rtocs_999_3,Night Training,"{'pairs': [['drive', 'loot'], ['lucky', 'glass...",1688791003356,1688791298267,"[nacho, five]",True,...,HIGH,1688791296700,"[alive, robot]",False,False,True,SAME,1688791297833,HIGH,1688791298267
6,6,64a7bcb2ba062572cd68791e,6423241561f11182a22e06c5,rtocs_999_3,Night Training,"{'pairs': [['claim', 'pour'], ['disc', 'spare'...",1688714131962,1688714418034,"[risk, shout]",False,...,HIGH,1688714416366,"[dress, joke]",False,True,True,DIFFERENT,1688714417666,HIGH,1688714418033
7,7,64a66338e409f9233fe1ba0c,6423241561f11182a22e06c5,rtocs_999_3,Night Training,"{'pairs': [['usual', 'plaza'], ['typo', 'pinto...",1688625669244,1688625976476,"[jade, type]",False,...,HIGH,1688625973593,"[expo, gall]",True,False,False,DIFFERENT,1688625975976,LOW,1688625976476


#### Export to New CSV

In [13]:
WPA_morning_testing_df.to_csv('../clean data/WPA updated/WPA_morning_testing_updated.csv')
WPA_night_training_df.to_csv('../clean data/WPA updated/WPA_night_training_updated.csv')